In [1]:
import pandas as pd
import numpy as np

# Set random seed for reproducibility
rng = np.random.default_rng(42)

# Generate synthetic data
n_samples = 1000
data = {
    'timestamp': pd.date_range(start='2024-01-01', periods=n_samples, freq='h'),
    'cpu_usage': rng.normal(50, 10, n_samples),       # CPU usage in percentage
    'memory_usage': rng.normal(60, 15, n_samples),    # Memory usage in percentage
    'network_latency': rng.normal(100, 20, n_samples), # Network latency in ms
    'disk_io': rng.normal(75, 10, n_samples),         # Disk I/O in MB/s
    'error_rate': rng.choice([0, 1], n_samples, p=[0.95, 0.05])  # 5% error rate
}

# Create DataFrame
df = pd.DataFrame(data)

# Display the first few rows of the dataset
print(df.head())


            timestamp  cpu_usage  memory_usage  network_latency    disk_io  \
0 2024-01-01 00:00:00  53.047171     59.110760        90.960980  87.490227   
1 2024-01-01 01:00:00  39.600159     49.060696        86.682446  81.876922   
2 2024-01-01 02:00:00  57.504512     53.782904       108.680196  94.661275   
3 2024-01-01 03:00:00  59.405647     69.508656       105.037088  59.521672   
4 2024-01-01 04:00:00  30.489648     60.044899        71.904169  72.346399   

   error_rate  
0           0  
1           0  
2           0  
3           0  
4           0  


In [2]:
from sklearn.ensemble import IsolationForest

# Implement anomaly detection using Isolation Forest
def detect_anomalies(data):
    model = IsolationForest(contamination=0.05, random_state=42)
    model.fit(data)
    anomalies = model.predict(data)
    return anomalies

# Detect anomalies in the dataset 
numeric_data = df.select_dtypes(include=[float, int]) # Only numeric columns 
df['anomaly'] = detect_anomalies(numeric_data)

print(df['anomaly'].value_counts()) # -1 denotes an anomaly

anomaly
 1    950
-1     50
Name: count, dtype: int64


In [3]:
from scipy.stats import zscore

# Calculate z-scores to identify anomalous values per column in anomalous rows
z_scores = numeric_data.apply(zscore)

# Function to identify anomalous columns for each row
def find_anomalous_columns(row, threshold=3):
    return [col for col in numeric_data.columns if abs(z_scores.loc[row.name, col]) > threshold]

# Apply the function to each anomalous row
df['anomalous_columns'] = df.apply(lambda row: find_anomalous_columns(row) if row['anomaly'] == -1 else [], axis=1)

# Display rows with anomalies and their anomalous columns
print(df[df['anomaly'] == -1][['timestamp', 'anomaly', 'anomalous_columns']])

              timestamp  anomaly anomalous_columns
26  2024-01-02 02:00:00       -1      [error_rate]
30  2024-01-02 06:00:00       -1      [error_rate]
53  2024-01-03 05:00:00       -1                []
76  2024-01-04 04:00:00       -1      [error_rate]
78  2024-01-04 06:00:00       -1      [error_rate]
83  2024-01-04 11:00:00       -1      [error_rate]
124 2024-01-06 04:00:00       -1      [error_rate]
136 2024-01-06 16:00:00       -1      [error_rate]
141 2024-01-06 21:00:00       -1         [disk_io]
142 2024-01-06 22:00:00       -1                []
164 2024-01-07 20:00:00       -1      [error_rate]
166 2024-01-07 22:00:00       -1      [error_rate]
176 2024-01-08 08:00:00       -1      [error_rate]
194 2024-01-09 02:00:00       -1      [error_rate]
208 2024-01-09 16:00:00       -1      [error_rate]
262 2024-01-11 22:00:00       -1      [error_rate]
269 2024-01-12 05:00:00       -1      [error_rate]
277 2024-01-12 13:00:00       -1                []
357 2024-01-15 21:00:00       -

In [4]:
from sklearn.tree import DecisionTreeClassifier

# Train a decision tree for root cause analysis
def root_cause_analysis(X_train, y_train, X_test):
    model = DecisionTreeClassifier(ccp_alpha=0.0, random_state=42)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    return predictions

# Prepare data for root cause analysis (only numeric columns)
X_train = df.drop(['anomaly', 'timestamp', 'anomalous_columns'], axis=1, errors='ignore')
y_train = df['anomaly']
predicted_causes = root_cause_analysis(X_train, y_train, X_train)
print(f"Predicted causes shape: {predicted_causes.shape}")

Predicted causes shape: (1000,)


In [5]:
# Example solution recommendation based on root cause
def recommend_solution(root_cause):
    solutions = {
        "network_error": "Restart the network service.",
        "database_issue": "Check the database connection and restart the service.",
        "high_cpu_usage": "Optimize running processes or allocate more resources."
    }
    return solutions.get(root_cause, "No recommendation available.")

# Recommend a solution based on a detected root cause
solution = recommend_solution("network_error")
print(f"Recommended solution: {solution}")

Recommended solution: Restart the network service.


In [6]:
# Simulate a network error by altering the dataset
df.loc[0, 'network_latency'] = 1000  # Simulating high network latency

# Prepare numeric data for prediction
numeric_test_data = df.drop(['anomaly', 'timestamp', 'anomalous_columns'], axis=1, errors='ignore')

# Run the troubleshooting agent
anomalies = detect_anomalies(df.select_dtypes(include=[float, int]))
predicted_causes = root_cause_analysis(X_train, y_train, numeric_test_data)
solution = recommend_solution(predicted_causes[0])
print(f"Detected issue: {predicted_causes[0]}")
print(f"Recommended solution: {solution}")

Detected issue: 1
Recommended solution: No recommendation available.
